In [1]:
import pickle
import pandas as pd 
import dask
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='16GB')

In [3]:
client

Client Scheduler: inproc://10.228.0.65/3433/1 Dashboard: http://10.228.0.65:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.00 GB


In [4]:
print (f'pickle version {pickle.format_version}')
print (f'Dask version {dask.__version__}')
print (f'Pandas version {pd.__version__}')

pickle version 4.0
Dask version 2.20.0
Pandas version 1.0.5


In [5]:
loaded_model = pickle.load(open("XgbClassBM4.pickle.dat", "rb"))

loaded_scaler = pickle.load(open ("ScalerBM4.pickle.dat" , "rb"))

In [6]:
loaded_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logitraw', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='approx', validate_parameters=1, verbosity=None)

In [7]:
def scaling_data_selected(X_test):

    selected_feat = ['AP_DFIRE2','AP_PISA','AP_T1','AP_T2','CP_MJ3h','SIPPER','ELE','VDW','PYDOCK_TOT','AP_dDFIRE']
    X_test = X_test[selected_feat]
    scaled_test = loaded_scaler.transform(X_test)

    return scaled_test

In [37]:
def load_data(path_to_data):
    """ here the data as csv file needs as Conformation ("Conf") column as index , to idenfy the decoy """
    try : 
            data = dd.read_csv(path_to_data)
            data = data.dropna()
            data = data.set_index('Conf')
#             print (len(data))

            return data

    except:
        print ("no data found")

        raise SystemExit()

In [38]:
X_test = load_data("../data/Clean_dataframe_unbalanced_scorers_set.csv")

In [19]:
X_test_scaled = scaling_data_selected(X_test)

In [30]:
my_index = X_test_scaled.index.compute()

In [21]:
prediction  = loaded_model.predict(X_test_scaled.to_dask_array())

In [22]:
# prediction.compute()

In [23]:
prediction_prob  = loaded_model.predict_proba (X_test_scaled.to_dask_array())

In [24]:
# prediction_prob.compute()

In [26]:
pred_results = [] 
for x,y in zip(prediction.compute(), prediction_prob.compute() ):
    pred_results.append((x,y))
#     print (f"{x} {y}")


In [31]:
pd.DataFrame(pred_results, columns=["Class","Prob"],index=my_index)

,Class,Prob
Conf,,
r_model-1000.pdb,0,-3.451228
r_model-1000.pdb,1,5.544319
r_model-1000.pdb,1,1.310003
r_model-1000.pdb,0,-1.055623
r_model-1000.pdb,0,-2.638464
...,...,...
r_model-3182.pdb,0,-2.539602
r_model-3183.pdb,0,-2.813540
r_model-3184.pdb,0,-2.912664
